<a href="https://colab.research.google.com/github/R4HUL-ROY/Multimodal_feature_extraction/blob/main/Glove_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary packages

In [1]:
import os
import urllib.request
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import pathlib

# Tobaco dataset Preparing

In [2]:
data_root = pathlib.Path('/content/drive/MyDrive/tobaco_OCR/')

print(data_root)
for item in data_root.iterdir():
  print(item)


/content/drive/MyDrive/tobaco_OCR
/content/drive/MyDrive/tobaco_OCR/Resume
/content/drive/MyDrive/tobaco_OCR/Memo
/content/drive/MyDrive/tobaco_OCR/Letter
/content/drive/MyDrive/tobaco_OCR/Report
/content/drive/MyDrive/tobaco_OCR/Email
/content/drive/MyDrive/tobaco_OCR/ADVE
/content/drive/MyDrive/tobaco_OCR/News
/content/drive/MyDrive/tobaco_OCR/Note
/content/drive/MyDrive/tobaco_OCR/Form
/content/drive/MyDrive/tobaco_OCR/Scientific


In [3]:
def get_file_paths_and_labels(data_root):
     text_paths = [str(path) for path in data_root.glob('*/*.txt')]
     labels = [p.split("/")[-2] for p in text_paths]
     return text_paths, labels

text_paths, labels = get_file_paths_and_labels(data_root)
print(text_paths)
print(labels)
print(len(text_paths))
print(len(labels))

['/content/drive/MyDrive/tobaco_OCR/Resume/50521422-1423.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50630631-0632.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/40028776-8777.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/10150247_10150256.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50538795-8796.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50201456-1467.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50719747-9748.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50264605-4625.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50590985-0986.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50296092-6093.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50535699-5699.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50239379-9380.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50467009-7010.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50425154-5155.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/40010133-0134.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50439450-9451.txt', '/c

# Utility Functions for preprocessing

In [4]:
import re
def preprocess_text(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = preprocessed_string.replace('_',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    return preprocessed_string

In [5]:
## Tokenize, Lemmatize, stopwords removal
import spacy 
import nltk
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words("english")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'count play home'

In [6]:
def get_text_from_path(path):
    with open(path) as f:
        lines = f.readlines()
        lines  = ' '.join(lines)
        f.close()
    return lines

out_text = get_text_from_path('/content/drive/MyDrive/tobaco_OCR/ADVE/0000435350.txt') 
# out_text = preprocess_text(out_text)
print(out_text)


 TE che fitm
 m66400 7127
 KOOLS are the only cigarettes that taste
 good when you have &® cold. They taste even
 better when you don't.
 Job No, K-2978
 ‘Mevapapars—300 iner—Mareh & April, 1956
 (5 9-4 in 4 108 ines) Pinel Proof (7) March 15, 1956



# Retrieve the actual texts from the file

In [7]:
texts= []
c= 0 
for i, this_path in enumerate(text_paths):
    texts.append(get_text_from_path(this_path))
    print(c, end= " ")
    c +=1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

# Create the dataframe

In [9]:
df = pd.DataFrame(list(zip(text_paths, texts,  labels)),
               columns =['text_path','texts', 'data_label'])
df.head()

,text_path,texts,data_label
0,/content/drive/MyDrive/tobaco_OCR/Resume/50521...,\n \n \n \n \n \n \n \n \n \n \n \n \n \n CURR...,Resume
1,/content/drive/MyDrive/tobaco_OCR/Resume/50630...,FF Principal investigator/Program Director (La...,Resume
2,/content/drive/MyDrive/tobaco_OCR/Resume/40028...,Name:\n Address:\n Date of Birth:\n Place of B...,Resume
3,/content/drive/MyDrive/tobaco_OCR/Resume/10150...,\n _ UNIVERSITY OF MIAMI\n CURRICULUM VITAE\n ...,Resume
4,/content/drive/MyDrive/tobaco_OCR/Resume/50538...,BIOGRAPHICAL SKETCH\n Provide the following in...,Resume


# Apply preprocessing utility function

In [10]:
df['texts'] = [preprocess_text(this_text) for this_text in df['texts']]
df.head()

,text_path,texts,data_label
0,/content/drive/MyDrive/tobaco_OCR/Resume/50521...,CURRICULUM VITAE Collaborating Investigator P...,Resume
1,/content/drive/MyDrive/tobaco_OCR/Resume/50630...,FF Principal investigatorProgram Director Last...,Resume
2,/content/drive/MyDrive/tobaco_OCR/Resume/40028...,Name Address Date of Birth Place of Birth Educ...,Resume
3,/content/drive/MyDrive/tobaco_OCR/Resume/10150...,UNIVERSITY OF MIAMI CURRICULUM VITAE Standard...,Resume
4,/content/drive/MyDrive/tobaco_OCR/Resume/50538...,BIOGRAPHICAL SKETCH Provide the following info...,Resume


In [11]:
df['texts'] = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in df['texts']]
df.head()

,text_path,texts,data_label
0,/content/drive/MyDrive/tobaco_OCR/Resume/50521...,curriculum vita collaborate investigator ptast...,Resume
1,/content/drive/MyDrive/tobaco_OCR/Resume/50630...,ff principal investigatorprogram director last...,Resume
2,/content/drive/MyDrive/tobaco_OCR/Resume/40028...,name address date birth place birth education ...,Resume
3,/content/drive/MyDrive/tobaco_OCR/Resume/10150...,university miami curriculum vita standard form...,Resume
4,/content/drive/MyDrive/tobaco_OCR/Resume/50538...,biographical sketch provide follow information...,Resume


In [58]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])

In [59]:
df.head()

,text_path,texts,data_label
0,/content/drive/MyDrive/tobaco_OCR/Resume/50521...,curriculum vita collaborate investigator ptast...,8
1,/content/drive/MyDrive/tobaco_OCR/Resume/50630...,ff principal investigatorprogram director last...,8
2,/content/drive/MyDrive/tobaco_OCR/Resume/40028...,name address date birth place birth education ...,8
3,/content/drive/MyDrive/tobaco_OCR/Resume/10150...,university miami curriculum vita standard form...,8
4,/content/drive/MyDrive/tobaco_OCR/Resume/50538...,biographical sketch provide follow information...,8


# Glove Model Downloading and Unzipping

In [12]:
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip')

('glove.6B.zip', <http.client.HTTPMessage at 0x7ff1f4a96250>)

In [13]:
!unzip "/content/glove.6B.zip" -d "/content/"

Archive:  /content/glove.6B.zip
  inflating: /content/glove.6B.50d.txt  
  inflating: /content/glove.6B.100d.txt  
  inflating: /content/glove.6B.200d.txt  
  inflating: /content/glove.6B.300d.txt  


In [41]:
emmbed_dict = {}
with open('/content/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector
f.close()

# Get the overall vocabulary of our dataset

In [30]:
our_vocab = []
max = 0
min = 0
for this_str in df['texts']:
    tmp = this_str.split(" ")
    if len(tmp) > max:
        max = len(tmp)
    if len(tmp) < min:
        min = len(tmp)    
    our_vocab.extend(tmp)

In [31]:
print(len(our_vocab))
print(our_vocab[1:100])

525016
['vita', 'collaborate', 'investigator', 'ptastname', '\ufeff1', 'name', 'imut', 'degroos', 'position', 'title', 'weisel', 'john', 'assoc', 'profof', 'cell', 'develp', 'biol', 'institution', 'locatio', 'swarthmore', 'college', 'swarthmore', 'pa', 'elect', 'engineer', 'marine', 'biol', 'laboratory', 'wood', 'hole', 'summer', '70', 'physiology', 'course', 'brandeis', 'university', 'waltham', 'phd', '1974', 'biophysics', 'embo', 'adv', 'course', 'cambridge', '9', 'fall', '73', 'image', 'process', 'england', 'position', 'hold', 'conclude', '1', 'present', 'position', 'list', 'chronological', 'orderali', 'position', 'hold', 'follow', 'baccalaureate', 'biophysics', 'gradstud', 'tinstitution', 'brandeis', 'university', 'dr', 'szent', '730873', 'gyorgyi', 'structural', 'postdoct', 'rosenstiel', 'basic', 'science', 'research', '197478', 'biology', 'fellow', 'center', 'brandeis', 'university', 'waltham', 'structural', 'senior', 'res', 'rosenstiel', 'basic', 'science', 'research', '197981',

In [36]:
print(max)
print(min)
print(len(set(our_vocab)))

986
0
77615


# Apply Glove embedding to our dataset

In [70]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_len = 1000

In [71]:
tokenizer = Tokenizer(num_words=len(df['texts']))
tokenizer.fit_on_texts(df['texts'])

In [72]:
text_sequence = tokenizer.texts_to_sequences(df['texts'])

In [73]:
text_padded = pad_sequences(
    text_sequence, maxlen = max_len, truncating = "post", padding = "post" 
)

In [74]:
word_index = tokenizer.word_index
len(word_index)

77612

In [75]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 200))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = emmbed_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [76]:
embedding_matrix.shape

(77613, 200)

# Train test Split

In [77]:
from sklearn.model_selection import train_test_split
label = df['data_label']
X_train, X_test, y_train, y_test = train_test_split(text_padded, label , test_size=0.2, random_state = 42)

In [78]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2785, 1000)
(2785,)
(697, 1000)
(697,)


In [79]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam


model = Sequential()

model.add(
    Embedding(
        num_words,
        200,
        embeddings_initializer = Constant(embedding_matrix),
        trainable = False
    )
)

# model.add(LSTM(200, dropout=0.1))
model.add(Dense(1024, activation="relu"))
# model.add(Dropout(0.2))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(10, activation="softmax"))

optimizer = Adam(learning_rate = 1e-3)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics = ["accuracy"])

In [80]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 200)         15522600  
                                                                 
 lstm_3 (LSTM)               (None, 200)               320800    
                                                                 
 dense_14 (Dense)            (None, 2048)              411648    
                                                                 
 dense_15 (Dense)            (None, 2048)              4196352   
                                                                 
 dense_16 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_17 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_18 (Dense)            (None, 512)              

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs = 20,
    validation_data = (X_test, y_test),
    verbose=1,
)

Epoch 1/20
88/88 [==============================] - 233s 3s/step - loss: 2.2211 - accuracy: 0.1630 - val_loss: 2.1594 - val_accuracy: 0.2123
Epoch 2/20
88/88 [==============================] - 228s 3s/step - loss: 2.1904 - accuracy: 0.1792 - val_loss: 2.1569 - val_accuracy: 0.1621
Epoch 3/20
88/88 [==============================] - 228s 3s/step - loss: 2.1841 - accuracy: 0.1688 - val_loss: 2.1484 - val_accuracy: 0.1722
Epoch 4/20
88/88 [==============================] - 237s 3s/step - loss: 2.1828 - accuracy: 0.1555 - val_loss: 2.1501 - val_accuracy: 0.2123
Epoch 5/20
88/88 [==============================] - 252s 3s/step - loss: 2.1833 - accuracy: 0.1623 - val_loss: 2.1512 - val_accuracy: 0.1722
Epoch 6/20
88/88 [==============================] - 252s 3s/step - loss: 2.1852 - accuracy: 0.1720 - val_loss: 2.1499 - val_accuracy: 0.2123
Epoch 7/20
88/88 [==============================] - 251s 3s/step - loss: 2.1826 - accuracy: 0.1558 - val_loss: 2.1504 - val_accuracy: 0.2123
Epoch 8/20
88